Imports and Paths

In [4]:
import kagglehub, pathlib, pandas as pd

path = pathlib.Path(kagglehub.dataset_download("martinellis/nhl-game-data"))
print("Path to dataset files:", path)


# See what files are available
list(path.glob("*.csv"))


Path to dataset files: C:\Users\puppy\.cache\kagglehub\datasets\martinellis\nhl-game-data\versions\2


[WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_goalie_stats.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_goals.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_officials.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_penalties.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_plays.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_plays_players.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_scratches.csv'),
 WindowsPath('C:/Users/puppy/.cache/kagglehub/datasets/martinellis/nhl-game-data/versions/2/game_sh

In [5]:
import pandas as pd

games = pd.read_csv(path / "game.csv", low_memory=False)
team_stats = pd.read_csv(path / "game_team_stats.csv", low_memory=False)
teams = pd.read_csv(path / "team_info.csv", low_memory=False)

games.shape, team_stats.shape, teams.shape


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\puppy\\.cache\\kagglehub\\datasets\\martinellis\\nhl-game-data\\versions\\2\\game_team_stats.csv'